# OCR Constitucion Arhuaco

In [8]:
import os
import re
import PyPDF2
import pandas as pd
from google.cloud import documentai
from google.api_core.client_options import ClientOptions
from typing import Optional, Sequence
from dotenv import load_dotenv

from google.api_core.client_options import ClientOptions
from google.cloud import documentai

load_dotenv()

True

In [9]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
endpoint = 'documentai.googleapis.com'
location = 'us'
project_id = os.getenv('project_id')
processor_id = os.getenv('processor_id')

## Dividir el PDF por partes

In [10]:
entrada_pdf = '/Users/juank/Dropbox/Mac/Desktop/workshop_nlp/data/Constitución de 1991_Ikun (Arhuaco).pdf'
salida_pdf = '/Users/juank/Dropbox/Mac/Desktop/workshop_nlp/data/Constitución_arhuaco_scrape.pdf'

cartas = {'carta_del_cabildo': {'esp': [68, 68], 'arh': [12, 12]}, 'palabras': {'esp': [69,70], 'arh': [13,14]}, 
          "cordinadores": {'esp': [71,72], 'arh': [15, 17]}, 'titulo_inicio': {'esp': [73, 73], 'arh': [18, 18]}}
constitucion = {'esp': [74, 82], 'arh': [[19, 33], [33, 47], [47, 58]]}

In [11]:
def extraer_paginas(entrada_pdf, salida_pdf, paginas):
    with open(entrada_pdf, 'rb') as archivo_entrada:
        lector_pdf = PyPDF2.PdfReader(archivo_entrada)
        
        escritor_pdf = PyPDF2.PdfWriter()
        
        for pagina_num in range (paginas[0], paginas[1] + 1):
            # Las páginas en PyPDF2 comienzan desde 0
            pagina = lector_pdf.pages[pagina_num - 1]
            escritor_pdf.add_page(pagina)
            
        with open(salida_pdf, 'wb') as archivo_salida:
            escritor_pdf.write(archivo_salida)
        return None

In [12]:
def get_pdf_to_text(salida_pdf):
    mime_type = 'application/pdf'   
    client = documentai.DocumentProcessorServiceClient(
    client_options=ClientOptions(api_endpoint=f"{location}-{endpoint}"))
    name = client.processor_path(project_id, location, processor_id)
    with open(salida_pdf, "rb") as image:
        image_content = image.read()
    
    raw_document = documentai.RawDocument(
        content=image_content, mime_type=mime_type)
    
    request = documentai.ProcessRequest(name=name, raw_document=raw_document)
    response = client.process_document(request=request)
    document = response.document
    return document

In [13]:
def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document"s text. This function converts
    offsets to a string.
    """
    
    return "".join(
        text[int(segment.start_index) : int(segment.end_index)]
        for segment in layout.text_anchor.text_segments
    )

In [14]:
def extract_text(document, text):
    lista_texto = []
    patron = r"\d+"
    ignore = ['Este texto es una traducción a partir del ikun\n', 'BIENVENIDO ARROYO\nCabildo Gobernador\n', "Emari kasteyano pari ikun re'zagikumana ni\n", 'Los artículos de la Constitución seleccionados para su traducción a la lengua ikun fueron reagrupados de\nacuerdo a distintos temas de interés para la población indígena. Esta reagrupación es el resultado de nuestro\ntrabajo; no corresponde siempre a la división en capitulos de la Constitución.\n']
    
    for pages in range(0, len(document.pages)):
        for blocks in range(0, len(document.pages[pages].blocks)):
            block = layout_to_text(document.pages[pages].blocks[blocks].layout, text)
            block = re.sub(r'G\.ij\.\.z\.\s(\d+)\.', r'GIJZ\1', block)
            if not re.match(patron, block) and block not in ignore:
                clean_block = [elemento.replace("\n", " ").replace("*", "").strip() for elemento in block.split(".") if elemento != "\n" and len(elemento.replace("\n", " ").replace("*", "").strip().split()) >= 4]
                #clean_block = clean_block.replace("\n", " ").strip()
                lista_texto.extend(clean_block)
    return lista_texto

In [15]:
def extract_tittles(document, text):
    lista_texto = []

    for pages in range(0, len(document.pages)):
        for blocks in range(0, len(document.pages[pages].blocks)):
            block = layout_to_text(document.pages[pages].blocks[blocks].layout, text)
            
            if block == '2. ARTÍCULOS RELATIVOS A LOS DERECHOS\nDE LAS PERSONAS\nArtículo 11. El derecho a la vida es inviolable. No habrá pena de muerte.\nArtículo 12. Nadie será sometido a desaparición forzada, a torturas ni a tratos o penas\ncrueles, inhumanos o degradantes.\n':
                lista_texto.append('2. ARTÍCULOS RELATIVOS A LOS DERECHOS\nDE LAS PERSONAS'.lower())
                
            block = re.sub(r'G\.ij\.\.z\.\s(\d+)\.', r'GIJZ\1', block).replace("\n", " ")
            clean_block = block.replace("\n", " ").replace("*", "")
            try:
                if clean_block.split('.')[1].strip().isupper() and clean_block.split('.')[0].isdigit():
                    lista_texto.append(clean_block.lower())
            except: pass
    return lista_texto

In [16]:
def extract_text_articles_arh (document, text, titulo_arh):
    patron = r'GIJZ(\d+)'
    grupo_parrafos = {}
    numero_anterior = None
    for pages in range(0, len(document.pages)):
            for blocks in range(0, len(document.pages[pages].blocks)):
                block = layout_to_text(document.pages[pages].blocks[blocks].layout, text)

                block = re.sub(r'G\.ij\.\.z\.\s(\d+)\.', r'GIJZ\1', block)
                block = re.sub(r'G\.i\.j\.\.z\.\s(\d+)\.', r'GIJZ\1', block)
                block = re.sub(r'G\.i\.j\.z\.\s(\d+)\.', r'GIJZ\1', block)
                block = re.sub(r'G\.ij\.z\.\s(\d+)\.', r'GIJZ\1', block)
                block = re.sub(r'G\.ij\-z\.\s(\d+)\.', r'GIJZ\1', block)

                resultado = re.match(patron, block)
                if resultado:
                    # Obtener el número del patrón
                    numero = resultado.group(1)
                    # Actualizar el número anterior
                    numero_anterior = numero
                    # Inicializar el contador de párrafos para este número si es la primera vez que lo encontramos
                    if numero not in grupo_parrafos:
                        block = block.replace("\n", " ").replace("*", "").replace(f"GIJZ{numero_anterior} ", "").replace("\'", "'").strip()
                        grupo_parrafos[numero] = [block.lower()]
                elif numero_anterior is not None and block != "\n" and len(block.replace("\n", " ").replace("*", "").strip().split()) >= 4 and block.replace("\n", " ").lower() not in titulo_arh:
                    # Incrementar el contador de párrafos para el número anterior
                    block = block.replace("\n", " ").replace("*", "").replace("\'", "").replace('G.i.j.z. ', '').strip()

                    grupo_parrafos[numero_anterior].append(block.lower())
    return grupo_parrafos

In [17]:
def extract_text_articles_esp (document, text, titulos_esp):
    patron = r'Artículo (\d+)'
    grupo_parrafos = {}
    numero_anterior = None
    for pages in range(0, len(document.pages)):
            for blocks in range(0, len(document.pages[pages].blocks)):
                block = layout_to_text(document.pages[pages].blocks[blocks].layout, text)
                
                if block == '2. ARTÍCULOS RELATIVOS A LOS DERECHOS\nDE LAS PERSONAS\nArtículo 11. El derecho a la vida es inviolable. No habrá pena de muerte.\nArtículo 12. Nadie será sometido a desaparición forzada, a torturas ni a tratos o penas\ncrueles, inhumanos o degradantes.\n':
                    grupo_parrafos['11'] = ['El derecho a la vida es inviolable. No habrá pena de muerte.'.lower()]
                    grupo_parrafos['12'] = ['Nadie será sometido a desaparición forzada, a torturas ni a tratos o penas crueles, inhumanos o degradantes.'.lower()]
                    
                resultado = re.match(patron, block)

                if resultado:
                    # Obtener el número del patrón
                    numero = resultado.group(1)
                    # Actualizar el número anterior
                    numero_anterior = numero
                    # Inicializar el contador de párrafos para este número si es la primera vez que lo encontramos
                    if numero not in grupo_parrafos:
                        block = block.replace("\n", " ").replace("*", "").replace(f"Artículo {numero_anterior}. ", "").strip()
                        grupo_parrafos[numero] = [block.lower()]
                elif numero_anterior is not None and block != "\n" and len(block.replace("\n", " ").replace("*", "").strip().split()) >= 4 and block.replace("\n", " ").lower() not in titulos_esp:
                    # Incrementar el contador de párrafos para el número anterior
                    block = block.replace("\n", " ").replace("*", "").replace('G.i.j.z. ', '').strip()

                    grupo_parrafos[numero_anterior].append(block.lower())
    return grupo_parrafos

In [18]:
df_final = pd.DataFrame()
for key in cartas.keys():
    paginas_a_extraer = cartas[key]['arh']
    # Llamar a la función para extraer las páginas
    escritor_pdf = extraer_paginas(entrada_pdf, salida_pdf, paginas_a_extraer)
    document_arh = get_pdf_to_text(salida_pdf)
    text_arh = document_arh.text
    lista_arh = extract_text(document_arh, text_arh)

    paginas_a_extraer = cartas[key]['esp']
    # Llamar a la función para extraer las páginas
    escritor_pdf = extraer_paginas(entrada_pdf, salida_pdf, paginas_a_extraer)
    document_esp = get_pdf_to_text(salida_pdf)
    text_esp = document_esp.text
    lista_esp = extract_text(document_esp, text_esp)
    
    # Unir datasets
    traduccion = []
    minimo = min(len(lista_arh), len(lista_esp))
    for i in range(0, minimo):
        traduccion.append({'esp': lista_esp[i], 'arh': lista_arh[i]})        
    df_traduccion = pd.DataFrame(traduccion)
    df_traduccion['parte'] = key
    
    df_final = pd.concat([df_final, df_traduccion], ignore_index=True)
df_final['esp'] = df_final['esp'].str.lower()
df_final['arh'] = df_final['arh'].str.lower()

In [19]:
df_final

,esp,arh,parte
0,carta del cabildo gobernador arhuaco,ikuzey sakuku umún a'zuna zuga'kunamu,carta_del_cabildo
1,"nabusímuke, mes de febrero del año 1994","nabusímuke, febrero timase' 1994 kugise'",carta_del_cabildo
2,"dias atrás, cuando se concluyó la elaboración ...","ema wa'nun, constitución abiti re'bori kuchuna...",carta_del_cabildo
3,"nosotros veíamos que de ser traducida, tal vez...","""eyméy nanundi, umún eyméy gwasi ki agwakunno ...",carta_del_cabildo
4,para ese trabajo se escogieron varias personas...,ey uwame' uya nikamu uzweingwasi i'ngwi agaguk...,carta_del_cabildo
...,...,...,...
62,muy satisfactorio también sentir la vitalidad ...,"uyéy a'chunakumana neykari, emi kartiyase' may...",cordinadores
63,a todos muchas gracias,"uyari, iku gumusinu awí zwein nugeykase', kwa ...",cordinadores
64,jon landaburu (cnrs) director del centro colom...,"constitución- se' a'nikwuya, yow nungwa, re'za...",cordinadores
65,"santafé de bogotá, febrero de 1994",iku diwún diwún akwey zoya enanuyáy winukunánu...,cordinadores


In [14]:
df_final.to_csv('/Users/frubana/Desktop/workshop_nlp/data/ocr_constitucion_arhuaco/constitucion_arh_cartas.csv', index=False)

In [15]:
articles_arh = {}
titulos_arh = []
for extract_pages in constitucion['arh']:
    paginas_a_extraer = extract_pages
    # Llamar a la función para extraer las páginas
    escritor_pdf = extraer_paginas(entrada_pdf, salida_pdf, paginas_a_extraer)
    document_arh = get_pdf_to_text(salida_pdf)
    text_arh = document_arh.text
    titulo_arh = extract_tittles(document_arh, text_arh)
    article_arh = extract_text_articles_arh (document_arh, text_arh, titulos_arh)
    
    titulos_arh.extend(titulo_arh)
    articles_arh.update(article_arh)
    


paginas_a_extraer = constitucion['esp']
# Llamar a la función para extraer las páginas
escritor_pdf = extraer_paginas(entrada_pdf, salida_pdf, paginas_a_extraer)
document_esp = get_pdf_to_text(salida_pdf)
text_esp = document_esp.text
titulos_esp = extract_tittles(document_esp, text_esp)
articles_esp = extract_text_articles_esp (document_esp, text_esp, titulos_esp)

In [16]:
traduccion = []
for i in range(0, len(titulos_esp)):
    traduccion.append({'esp': titulos_esp[i].strip(), 'arh': titulos_arh[i].strip()})        
df_titulos = pd.DataFrame(traduccion)
df_titulos['parte'] = 'titulos_constitucion'
df_final = pd.concat([df_final, df_titulos], ignore_index=True)

In [17]:
df_titulos

,esp,arh,parte
0,1. artículos relativos a los principios fundam...,1. kuma'pinre'tanazey nari kutukunun kutu abun...,titulos_constitucion
1,2. artículos relativos a los derechos\nde las ...,2. i'ngweti i'ngweti menanuyáy mikunánuya neyk...,titulos_constitucion
2,3. artículos relativos a la protección de los ...,3. tanu me'zari kwey awkwa neyka enanuyáy íkwuya,titulos_constitucion
3,4. artículos relativos a la nacionalidad,4. akwey zoyeyka enanuyáy jwa'kumukweyna kunar...,titulos_constitucion
4,5. artículos relativos a los derechos de las l...,"5. niwi ka' jomu riwun, diwún diwún aséykumey,...",titulos_constitucion
5,6. artículos relativos a la participación en l...,6. ga'kunamu riñawazori awungwa neyka zaka' un...,titulos_constitucion
6,7. artículo relativo a la justicia en los pueb...,"7. iku diwún diwún akwey zoya'ba, gwamu ungwak...",titulos_constitucion
7,8. artículos relativos a la organización de lo...,"7. iku diwún diwún akwey zoya'ba, gwamu ungwak...",titulos_constitucion
8,9. artículos relativos a los planes de desarrollo,"8. ka'gumu re'kumey zoyana, wasáy, du zákumey,...",titulos_constitucion


In [18]:
traduccion = []
for key in articles_esp.keys():
    try:
        blocks_esp = articles_esp[key]
        blocks_arh = articles_arh[key]
        minimo = min(len(blocks_esp), len(blocks_arh))
        for i in range(0, minimo):
            traduccion.append({'esp': blocks_esp[i], 'arh': blocks_arh[i]})
    except:
        pass
df_articulos = pd.DataFrame(traduccion)
df_articulos['parte'] = 'articulos'
df_final = pd.concat([df_final, df_articulos], ignore_index=True)
df_articulos

,esp,arh,parte
0,"colombia es un estado social de derecho, organ...","koronbia awgeykari, i'ngwi nánukin na ni. ey u...",articulos
1,son fines esenciales del estado: servir a la c...,"kumu a'nikwuya pinnazey nari, i'ngwi nánukin r...",articulos
2,las autoridades de la república están institui...,"niwi ka jomu riwun sakuku agagusi chokumanari,...",articulos
3,la soberanía reside exclusivamente en el puebl...,"akín kwey zorikureyka, bema gwasi umún a'zuna ...",articulos
4,2. artículos relativos a los derechos de las p...,gwamu guga unchuiiuya gwamu guga unchunuya rez...,articulos
...,...,...,...
74,habrá un consejo nacional de planeación integr...,kuma'pinre'tana yamun yamun inu úkumey wazwein...,articulos
75,los miembros del consejo nacional serán design...,ey uweki gwamu guga unchunun gwásuya sakuku um...,articulos
76,en las entidades territoriales habrá también c...,"eyma jina nanuyeykari, riñawazori awungwa neyk...",articulos
77,el consejo nacional y los consejos territorial...,"kagumu kumapinretanase resana naba ayeygwi, in...",articulos


In [19]:
df_final

,esp,arh,parte
0,carta del cabildo gobernador arhuaco,ikuzey sakuku umún a'zuna zuga'kunamu,carta_del_cabildo
1,"nabusímuke, mes de febrero del año 1994","nabusímuke, febrero timase' 1994 kugise'",carta_del_cabildo
2,"dias atrás, cuando se concluyó la elaboración ...","ema wa'nun, constitución abiti re'bori kuchuna...",carta_del_cabildo
3,"nosotros veíamos que de ser traducida, tal vez...","""eyméy nanundi, umún eyméy gwasi ki agwakunno ...",carta_del_cabildo
4,para ese trabajo se escogieron varias personas...,ey uwame' uya nikamu uzweingwasi i'ngwi agaguk...,carta_del_cabildo
...,...,...,...
150,habrá un consejo nacional de planeación integr...,kuma'pinre'tana yamun yamun inu úkumey wazwein...,articulos
151,los miembros del consejo nacional serán design...,ey uweki gwamu guga unchunun gwásuya sakuku um...,articulos
152,en las entidades territoriales habrá también c...,"eyma jina nanuyeykari, riñawazori awungwa neyk...",articulos
153,el consejo nacional y los consejos territorial...,"kagumu kumapinretanase resana naba ayeygwi, in...",articulos


In [21]:
lexico = {'patrimonio arqueolégico': 'Birin zanv pinnazey nari inwki eyki chuzdéneya',
          'diversidad étnica y cultural': "Diwun diwun winnenanuyay akwuya",
          'grupos étnicos': 'Diwan diwan winnenanuydy akwey zoya',
          'minorias politicas': "Diwén diwan arufiey awa re'muga re'béakumey zoya",
          'consejero': "Du kawi ga'kunamu wésuya",
          'derecho': 'Enanuyay',
          'parques naturales': "Eytiwe ikwey ka'gume re‘kumana",
          'Constitucién Politica': "Ga'kanama abunna zukuta",
          'titulo': "Ga'kunamy sumy neyka zaka'chésuya",
          'capitulo':"Ga'kunamy sump na'ba agagiikuya",
          'articulo': "G.ij.z. (Ga'kuname i'ngwijuna zaka'chésuya)",
          'articulo transitorio': 'G.ij.z. mend ndnukin wazweingwa neyka',
          'ley': 'Gwamy guga',
          'senador': "Gwamw guga guwa umin a'zuna",
          'rama legislativa': 'Gwamwu guga gawuydrigun',
          'cuerpo legislativo o congresistas': 'Gwamu guga gawuya-jina',
          'Representante a la Camara': "Gwamua guga guwa awd a'zuna",
          'Ley Organica': 'Gwamu guga zukuta',
         'Ley Organica de Ordenamiento Territorial': "Gwamwu guga zukuta ka'gumu re'kumey zoya si wasuya",
         'cuerpo ejecutivo': 'Gwamy yeyka unchufiwn gwdsuyajina',
         'decreto': 'Gwamw guga unchuitdsuya',
         'resolucién': "Gwamu guga unchuitwya re'zugurusi zoya",
         'los derechos de las personas': "I'ngweti i‘ngweti menanuydy mikundnuya neyka",
         'consejo consultivo': 'Inu ikumey zweingwa du kawi ga‘kanama wésuyajina',
         'regalias': "Ka'se' inu a'chuiakumuyeykazey nari,jwi anni'kumyyari",
         'reunién de municipios (provincia)': "Ka'gumy awd re'kumana'ba windiiunni'sana",
         'territorio indigena': "Ka'gume fkuzey re'kumana",
         'Comision de Ordenamiento Territorial': "Ka'guma re‘kumungwa'ba chwi winzwein nikuyajina",
         'estado': "Kuma'pinre’tana",
         'cédula de ciudadania': "Jwa'kumukweyna",
         'indocumentado': "Jwa'kumukweyna kuna’ neyka",
         'accion de tutela': 'Michwi Enanuyay Winikwuya',
         'persona con doble nacionalidad': "Mu'bajwa'kumukwéy neyka kundnuya",
         'nuestro pais (Colombia)': "Niwi ka'-jome riwen",
         'nuestro territorio (Linea Negra)': "Niwi U'meke",
         'cuerpo jurisdiccional (juez)': "No'ku' neykazey, a'tuhuyeykazey gwame gwdkuya",
         'departamento': "(Niwi ka'jome riwun) ka'gumy re'kumana  umiin zauna",
         'municipio': "(Niwi ka'jome riwen) ka'gumy re'kumana awuzuna",
         'empleado publico': "Re'masi neyka ka'mukanungwasi iku a'chuhakumana",
         'accion popular': "Re'masi Neykazey Nari Nanaydy Winikwuya",
         'plan o programa de desarrollo': 'Rifawazori awungwa neyka',
         'concejales':  "Sakuku dikin wina'zuna awa re'kumana'ba zanu",
         'miembros de las asambleas departamentales': "Sakuku dikin wina'zana umért awutari re'kumana'ba zane",
         'autoridad del departamento (Gobernador)': "Sakuku ka'gume umn awutari re'kumana'ba zanu",
         'autoridad del municipio (Alcalde)': "Sakuku ka'gume awd re‘kumana'ba zanu",
         'ciudadano': "Umi a'zuna-jwa'kumukwéy neyka kundneya"
         }

In [28]:
corpus_constitucion = []

for key_w, value_e in lexico.items():
    corpus_constitucion.append({'arh': value_e, 'esp': key_w, 'parte': 'diccionario'})
df_diccionario = pd.DataFrame(corpus_constitucion)
df_final = pd.concat([df_final, df_diccionario], ignore_index=True)
df_diccionario

,arh,esp,parte
0,Birin zanv pinnazey nari inwki eyki chuzdéneya,patrimonio arqueolégico,diccionario
1,Diwun diwun winnenanuyay akwuya,diversidad étnica y cultural,diccionario
2,Diwan diwan winnenanuydy akwey zoya,grupos étnicos,diccionario
3,Diwén diwan arufiey awa re'muga re'béakumey zoya,minorias politicas,diccionario
4,Du kawi ga'kunamu wésuya,consejero,diccionario
5,Enanuyay,derecho,diccionario
6,Eytiwe ikwey ka'gume re‘kumana,parques naturales,diccionario
7,Ga'kanama abunna zukuta,Constitucién Politica,diccionario
8,Ga'kunamy sumy neyka zaka'chésuya,titulo,diccionario
9,Ga'kunamy sump na'ba agagiikuya,capitulo,diccionario


In [29]:
df_final.to_csv('/Users/frubana/Desktop/workshop_nlp/data/ocr_constitucion_arhuaco/constitucion_arh_completo.csv', index=False)

In [30]:
df_final

,esp,arh,parte
0,carta del cabildo gobernador arhuaco,ikuzey sakuku umún a'zuna zuga'kunamu,carta_del_cabildo
1,"nabusímuke, mes de febrero del año 1994","nabusímuke, febrero timase' 1994 kugise'",carta_del_cabildo
2,"dias atrás, cuando se concluyó la elaboración ...","ema wa'nun, constitución abiti re'bori kuchuna...",carta_del_cabildo
3,"nosotros veíamos que de ser traducida, tal vez...","""eyméy nanundi, umún eyméy gwasi ki agwakunno ...",carta_del_cabildo
4,para ese trabajo se escogieron varias personas...,ey uwame' uya nikamu uzweingwasi i'ngwi agaguk...,carta_del_cabildo
...,...,...,...
196,concejales,Sakuku dikin wina'zuna awa re'kumana'ba zanu,diccionario
197,miembros de las asambleas departamentales,Sakuku dikin wina'zana umért awutari re'kumana...,diccionario
198,autoridad del departamento (Gobernador),Sakuku ka'gume umn awutari re'kumana'ba zanu,diccionario
199,autoridad del municipio (Alcalde),Sakuku ka'gume awd re‘kumana'ba zanu,diccionario
